# Scaling Panel apps with Dask

By combining Panel and Dask you can **scale your apps to bigger datasets, bigger calculations and more users**.

Compared to other data app frameworks Panel is novel in that Panel is a reactive framework that supports async operations. This means you can **off load large computations to your Dask cluster and update the Panel app as the results are ready**.

This makes Panel and Dask a perfect match.

## Async/Await and Non-Blocking Execution

This approach is great if you want to offload larger computations to Dask while keeping your Panel data app responsive. Please note that off loading the computations to the Dask client comes with some cost for transfering the input and output data

You might find additional inspiration in the [Panel Async and Concurrency Guide](Async_and_Concurrency.ipynb) and the [Dask Async/Await and Non-Blocking Execution Documentation](https://examples.dask.org/applications/async-await.html#Async/Await-and-Non-Blocking-Execution).

Lets start by installed *dask distributed*

```bash
pip install dask[distributed]
```

We will define the backgrounds tasks in a seperate file called `tasks.py`. Please note not that both the Panel app and the Dask Cluster needs to be able to `import tasks`.

**tasks.py**

```python
import time
import numpy as np

def blocking_computation(x: float) -> float:
    samples = []
    for _ in range(1000):
        time.sleep(0.001)
        samples.append(np.random.normal(loc=1.0, scale=1.0))
    result = x + int(np.ceil(np.mean(samples)))
    return result
```

Lets define the Panel app

**app.py**

```python
# dask_example.py
from datetime import datetime as dt

import param

from dask.distributed import Client
from tasks import blocking_computation

import panel as pn

pn.extension()

@pn.cache
def get_client():
    return Client()

get_client() # For some unknown reason this is needed. Otherwise we get ModuleNotFoundError: No module named 'tasks'

async def submit(func, *args, **kwargs):
    client = get_client()
    future = client.submit(func, *args, **kwargs)
    return await client.gather(future, asynchronous=True)

float_input = pn.widgets.FloatInput(value=0, name="Input")
submit_button = pn.widgets.Button(name="Run in background", button_type="primary")
other_widget = pn.widgets.IntSlider(value=0, start=0, end=10, name="Non blocked slider")
pn.Column(float_input, submit_button, other_widget, other_widget.param.value).servable()

def start():
    submit_button.disabled=True
    float_input.disabled=True
    float_input.loading=True

def stop():
    float_input.disabled=False
    float_input.loading=False
    submit_button.disabled=False

@pn.depends(submit_button, watch=True)
async def run(_):
    start()
    float_input.value = await submit(blocking_computation, float_input.value)
    stop()
```

**Please note we using a synchronous Client and `gather` `asynchronous` simply because I cannot get `Client(asynchronous=True)` working with Panel.**

**Please note off loading this computation to Dask seems to add an overhead of +10sconds for transfering the result from Dask back to Panel???** If I change the slider the transfer is immediate!!!!